# Model Training
This notebook defines and trains the neural network model for fire risk classification.

## Import Required Libraries

In [ ]:
import torch
from torch import nn, optim

## Load Preprocessed Data
Note: Run the preprocessing notebook first to generate these tensors

In [ ]:
# Load the preprocessed data from the previous notebook
# X_train_t, y_train_t, X_val_t, y_val_t, X_test_t, y_test_t should be available
# If running independently, uncomment and run the preprocessing notebook first

print("Preprocessed data should be loaded from the preprocessing notebook")

## Define Neural Network Architecture

In [ ]:
class RiskClassifier(nn.Module):
    def __init__(self, input_size, hidden_size=64, output_size=3):
        super(RiskClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size//2)
        self.fc3 = nn.Linear(hidden_size//2, output_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = self.fc1(x)
        #x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        #x = self.relu(x)
        x = self.dropout(x)
        x = self.fc3(x)
        return x

print("RiskClassifier model defined successfully")

## Initialize Model, Loss Function, and Optimizer

In [ ]:
# Initialize model, loss, optimizer
model = RiskClassifier(X_train_t.shape[1])
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

print("Model initialized")
print(f"Input size: {X_train_t.shape[1]}")
print(f"Model:\n{model}")

## Training Loop

In [ ]:
# Training loop
epochs = 30
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_t)
    loss = criterion(outputs, y_train_t)
    loss.backward()
    optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val_t)
        val_loss = criterion(val_outputs, y_val_t)
        val_pred = val_outputs.argmax(dim=1)
        acc = (val_pred == y_val_t).float().mean()
    
    if (epoch + 1) % 5 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {loss.item():.4f}, "
              f"Val Loss: {val_loss.item():.4f}, Val Acc: {acc.item():.4f}")

print("\nTraining completed")